In [1]:
import torch.nn.functional as F
from torch.utils import data
import torch
import os
import random
import numpy as np

indice1=300
sample=32
cl=16
indice1 = 300

train = np.load("train.npy")
train = torch.from_numpy(np.stack(train, 0))
test = np.load("test.npy")
test = torch.from_numpy(np.stack(test, 0))
valid = np.load("valid.npy")
valid = torch.from_numpy(np.stack(valid, 0))

print(train.shape)
print(test.shape)
print(valid.shape)

label=np.arange(0,cl,1)
label = label.astype(np.float32)
train_label = torch.from_numpy(np.tile(label, indice1)) 
test_label  = torch.from_numpy(np.tile(label, 100))
valid_label = torch.from_numpy(np.tile(label, 2))

print(train_label.dtype)
print(train_label.shape)
print(test_label.shape)
print(valid_label.shape)


from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

train_dataset = TensorDataset(train, train_label)
valid_dataset = TensorDataset(valid, valid_label)
test_dataset = TensorDataset(test, test_label)
batch=cl*2

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch,            
    shuffle= True,          
)

valid_loader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch,           
    shuffle= True,          
)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = batch,     
    shuffle= True,            
)

torch.Size([4800, 32, 204])
torch.Size([1600, 32, 204])
torch.Size([32, 32, 204])
torch.float32
torch.Size([4800])
torch.Size([1600])
torch.Size([32])


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse

from torch.autograd import Variable
import torch.optim as optim
from torch.utils import data
import os
import random


parser = argparse.ArgumentParser(description='Sequence Modeling')
parser.add_argument('--batch_size', type=int, default=cl*2, metavar='N',
                    help='batch size (default: 128)')
parser.add_argument('--cuda', action='store_true', default=False,
                    help='Disable cuda for training')
parser.add_argument('--dropout', type=float, default=0.45,
                    help='dropout applied to layers (default: 0.05)')
parser.add_argument('--clip', type=float, default=0.35,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=30,
                    help='upper epoch limit (default: 20)')
parser.add_argument('--levels', type=int, default=4,
                    help='# of levels (default: 8)')
parser.add_argument('--log-interval', type=int, default=1, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--lr', type=float, default=1e-4,
                    help='initial learning rate (default: 2e-3)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--seq_len', type=int, default=204)

args = parser.parse_args([])

if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
print(args)

torch.manual_seed(args.seed)
batch_size = args.batch_size  
print(batch_size)
epochs = args.epochs 
input_channels = sample
print(input_channels)
channel_sizes = [input_channels*2] * args.levels   
n_classes = cl
print(n_classes)
steps = 0


import math
from einops import rearrange
from torch.nn import init
from torch.nn.utils import weight_norm

class ConvNet(nn.Module):
    def __init__(self, s=64, p=64, l=204):
        super(ConvNet, self).__init__()

        self.tconv1 = weight_norm(nn.Conv1d(s, p, kernel_size=3,stride=1, padding=1, groups=s))
        self.p1 = nn.Conv1d(p, p, kernel_size=1, stride=1)
        self.relu1 = nn.ReLU()
        self.tconv2 = weight_norm(nn.Conv1d(p, p, kernel_size=3,stride=1, padding=1, groups=p))
        self.p2 = nn.Conv1d(p, p, kernel_size=1, stride=1)
        self.relu2 = nn.ReLU()
        self.tconv3 = weight_norm(nn.Conv1d(p, p, kernel_size=3,stride=1, padding=1, groups=p))
        self.p3 = nn.Conv1d(p, p, kernel_size=1, stride=1)
        self.relu3 = nn.ReLU()
        self.tconv4 = weight_norm(nn.Conv1d(p, p, kernel_size=3,stride=1, padding=1, groups=p))
        self.p4 = nn.Conv1d(p, p, kernel_size=1, stride=1)
        self.relu4 = nn.ReLU()
        self.ln0 = nn.LayerNorm([p,l],elementwise_affine = True)
        self.init_weights()
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()

    def init_weights(self):
        self.tconv1.weight.data.normal_(0, 0.01)
        self.tconv2.weight.data.normal_(0, 0.01)
        self.tconv3.weight.data.normal_(0, 0.01)
        self.tconv4.weight.data.normal_(0, 0.01)

    def forward(self, x):

        out_tcn1 = self.relu1(self.p1(self.tconv1(x)))
        out_tcn2 = self.relu2(self.p2(self.tconv2(out_tcn1)))
        out_tcn3 = self.relu3(self.p3(self.tconv3(out_tcn2)))
        out = self.ln0(self.relu4(self.p4(self.tconv4(out_tcn3)))) 
        out =  self.relu(self.dropout(out))

        return out
class Moblienet(nn.Module):
    def __init__(self, input_size, output_size,p,l):
        super(Moblienet, self).__init__()

        self.length=l
        self.tcn = ConvNet(input_size,p,l)
        self.linear = nn.Linear(l*4, output_size)

    def forward(self, x):
        out= self.tcn(x)

        y2 = F.adaptive_avg_pool2d(out.permute(0, 2, 1), [self.length,4])
        y3 = y2.view(y2.size(0), -1)
        o = self.linear(y3)

        o = F.log_softmax(o, dim=1).contiguous()

        return o

Namespace(batch_size=32, cuda=False, dropout=0.45, clip=0.35, epochs=30, levels=4, log_interval=1, lr=0.0001, optim='Adam', seed=1111, seq_len=204)
32
32
16


In [6]:
from torchsummary import summary

model = Moblienet(input_channels, n_classes, (sample//4)*8, 51*4)
if args.cuda:
    model.cuda()

summary(model, input_size=[(input_channels, 204)], batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [32, 64, 204]             256
            Conv1d-2              [32, 64, 204]           4,160
              ReLU-3              [32, 64, 204]               0
            Conv1d-4              [32, 64, 204]             256
            Conv1d-5              [32, 64, 204]           4,160
              ReLU-6              [32, 64, 204]               0
            Conv1d-7              [32, 64, 204]             256
            Conv1d-8              [32, 64, 204]           4,160
              ReLU-9              [32, 64, 204]               0
           Conv1d-10              [32, 64, 204]             256
           Conv1d-11              [32, 64, 204]           4,160
             ReLU-12              [32, 64, 204]               0
        LayerNorm-13              [32, 64, 204]          26,112
          Dropout-14              [32, 

In [7]:
import torchmetrics
from sklearn.metrics import confusion_matrix

lr = args.lr
optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)

losses = []
loss_test = []
accuracy = []
fm = []
atts= []

acc_valid = []


def train(ep):
    train_loss = 0
    model.train()
    for batch_idx, (bands, target) in enumerate(train_loader):
 #       if args.cuda: bands, target = bands.cuda(), target.cuda()
        bands, target = Variable(bands), Variable(target)
        optimizer.zero_grad()
        output = model(bands)
        loss = F.nll_loss(output, target.long())
        loss.backward()
        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        train_loss += loss
        if batch_idx % args.log_interval == 0:
            los = train_loss.item()/args.log_interval
            losses.append(los)
            train_loss = 0

def test(loader):
    model.eval()
    test_acc = torchmetrics.Accuracy()
    test_fm = torchmetrics.ConfusionMatrix(threshold=1./16,num_classes=16)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for bands, target in loader:
#            if args.cuda: bands, target = bands.cuda(), target.cuda()
            with torch.no_grad():
                bands, target = Variable(bands), Variable(target)
            pred = model(bands)
            test_loss += F.nll_loss(pred, target.long()).item()
            correct += (pred.argmax(1) == target.long()).type(torch.float).sum().item()
            test_acc(pred.argmax(1), target.long())
            test_fm(pred.argmax(1), target.long())

    test_loss /= len(test_loader)
    loss_test.append(test_loss)
    correct /= len(test_loader.dataset)

    total_acc = test_acc.compute()
    total_acc = torch.mean(total_acc)
    accuracy.append(total_acc)

    total_fm = test_fm.compute()
    fm.append(total_fm)

       
    print(f"Accuracy: {(100 * correct):>0.1f}%, "f"Avg loss: {test_loss:>8f}, "f"torch metrics acc: {total_acc}%")

    test_fm.reset()

    return test_loss

def valid(loader):
    model.eval()
    test_acc = torchmetrics.Accuracy()
    test_fm = torchmetrics.ConfusionMatrix(threshold=1./16,num_classes=16)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for bands, target in loader:
#            if args.cuda: bands, target = bands.cuda(), target.cuda()
            with torch.no_grad():
                bands, target = Variable(bands), Variable(target)
            pred = model(bands)
            test_loss += F.nll_loss(pred, target.long()).item()
            correct += (pred.argmax(1) == target.long()).type(torch.float).sum().item()
            test_acc(pred.argmax(1), target.long())
            test_fm(pred.argmax(1), target.long())

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    total_acc = test_acc.compute()
    acc_valid.append(total_acc)

    total_fm = test_fm.compute()
    fm.append(total_fm)
       
    print(f"Valid Accuracy: {(100 * correct):>0.1f}%, "f"Avg loss: {test_loss:>8f}, "f"torch metrics acc: {total_acc}%")

    test_acc.reset()
    test_fm.reset()

    return test_loss

if __name__ == "__main__":

    for epoch in range(1, epochs+1):
        train(epoch)
        if epoch%5==0:
            print("Epoch: ",epoch)
            print("TEST:") 
            test(test_loader)
#            print("VALID:")
#            valid(valid_loader)
            print("\n")
            
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr 

Epoch:  5
TEST:
Accuracy: 47.2%, Avg loss: 1.566535, torch metrics acc: 0.4724999964237213%


Epoch:  10
TEST:
Accuracy: 63.1%, Avg loss: 1.013310, torch metrics acc: 0.6312500238418579%


Epoch:  15
TEST:
Accuracy: 65.8%, Avg loss: 0.972509, torch metrics acc: 0.6581249833106995%


Epoch:  20
TEST:
Accuracy: 66.6%, Avg loss: 0.935869, torch metrics acc: 0.6662499904632568%


Epoch:  25
TEST:
Accuracy: 66.9%, Avg loss: 0.931921, torch metrics acc: 0.6693750023841858%


Epoch:  30
TEST:
Accuracy: 67.2%, Avg loss: 0.927736, torch metrics acc: 0.671875%


